In [ ]:
%%configure -f
{
    "conf": {
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
    }
}

In [ ]:
sc.install_pypi_package('boto3', 'https://pypi.org/simple')

In [4]:
import boto3
from concurrent.futures import ThreadPoolExecutor 
from functools import reduce
from pyspark.sql import DataFrame

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
data = spark.read.parquet('s3://wos-bucket/PurchaseOrder.xml.parquet')
data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- purchaseOrder: struct (nullable = true)
 |    |-- purchaseOrder@orderDate: timestamp (nullable = true)
 |    |-- shipTo: struct (nullable = true)
 |    |    |-- shipTo@country: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- street: string (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- state: string (nullable = true)
 |    |    |-- zip: double (nullable = true)
 |    |-- billTo: struct (nullable = true)
 |    |    |-- billTo@country: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- street: string (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- state: string (nullable = true)
 |    |    |-- zip: double (nullable = true)
 |    |-- comment: string (nullable = true)
 |    |-- items: struct (nullable = true)
 |    |    |-- item: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- item@partNu

In [9]:
par_files = ['s3://wos-bucket/PurchaseOrder.xml.parquet',
             's3://wos-bucket/p2.parquet',
             's3://wos-bucket/p3.parquet']

# Define read function
def from_s3(file):
    return spark.read.parquet(file)
    
# Read test Parquet files from S3 in parallel
with ThreadPoolExecutor(max_workers = 3) as executor:
    results = executor.map(from_s3, par_files)
    
df1, df2, df3 = results

# Combine files into a single dataframe
# From https://stackoverflow.com/questions/37612622/spark-unionall-multiple-dataframes
dfs = [df1, df2, df3]
new_data = reduce(DataFrame.unionAll, dfs)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…